In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *


In [18]:
nodes = [1, 2, 3]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-3/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 140
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,137.7m,0.094,1.806,3.4e-04,3.6e-04,1.3e-04,nan,nan°
2by1,129.9m,0.115,1.851,3.2e-04,5.1e-04,6.4e-05,nan,nan°
3by1,134.4m,0.146,3.290,5.7e-04,6.2e-04,1.3e-04,nan,nan°
Avg.,134.0,0.118,2.32,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.090,0.073,0.041",0.122,3.04°,"-0.023,+0.014,-0.022",-1.87°
1->3,"0.073,0.080,0.025",0.112,2.77°,"-0.029,-0.032,-0.007",+0.28°
Avg:,,0.117,2.91°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,138.0m,0.091,1.658,9.7e-05,9.3e-05,3.4e-05,nan,nan°
2by1,129.8m,0.089,1.053,8.1e-05,8.6e-05,1.8e-05,nan,nan°
3by1,133.1m,0.328,5.601,7.3e-04,3.3e-03,3.7e-05,nan,nan°
Avg.,133.6,0.169,2.77,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.059,0.062,0.048",0.098,1.67°,"-0.007,+0.018,-0.036",-0.72°
1->3,"0.216,0.216,0.035",0.308,5.38°,"0.055,-0.089,-0.029",+3.66°
Avg:,,0.203,3.53°,,


In [17]:
nodes = [1, 2, 3, 4]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-4/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 110
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,114.1m,0.098,2.046,2.9e-04,2.5e-04,1.8e-04,nan,nan°
2by1,108.4m,0.146,3.497,4.7e-04,2.5e-04,4.4e-04,nan,nan°
3by1,113.3m,0.119,2.576,4.2e-04,3.3e-04,1.0e-04,nan,nan°
4by1,67.7m,0.096,1.623,2.7e-04,5.5e-04,3.5e-04,nan,nan°
Avg.,100.9,0.115,2.44,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.051,0.045,0.067",0.096,2.06°,"-0.005,+0.008,-0.041",-0.08°
1->3,"0.067,0.050,0.026",0.088,1.96°,"-0.027,-0.001,-0.000",+0.13°
1->4,"0.027,0.029,0.038",0.055,1.15°,"-0.009,+0.009,-0.016",+0.16°
Avg:,,0.080,1.72°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,114.8m,0.086,1.625,9.4e-05,8.8e-05,3.7e-05,nan,nan°
2by1,108.1m,0.082,1.103,8.7e-05,9.3e-05,1.9e-05,nan,nan°
3by1,112.2m,0.322,5.454,3.9e-04,1.9e-03,2.3e-05,nan,nan°
4by1,67.4m,0.154,3.023,6.3e-04,7.2e-04,1.0e-04,nan,nan°
Avg.,100.6,0.161,2.80,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.063,0.066,0.040",0.099,1.74°,"-0.008,+0.023,-0.027",-0.89°
1->3,"0.210,0.214,0.036",0.302,4.74°,"0.046,-0.090,-0.032",+3.75°
1->4,"0.054,0.065,0.040",0.094,1.12°,"-0.003,-0.027,+0.037",-0.28°
Avg:,,0.165,2.53°,,


In [12]:
nodes = [1, 2, 3, 4, 5]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-5/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 110
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,114.4m,0.091,1.600,2.8e-04,2.7e-04,2.7e-04,nan,nan°
2by1,110.2m,0.121,1.693,1.0e-03,2.1e-04,3.9e-04,nan,nan°
3by1,115.9m,0.131,2.391,9.8e-04,1.3e-03,2.6e-04,nan,nan°
4by1,69.5m,0.110,2.060,3.6e-04,2.6e-04,2.8e-04,nan,nan°
5by1,107.8m,0.125,1.622,2.0e-03,1.2e-03,1.8e-03,nan,nan°
Avg.,103.5,0.116,1.87,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.060,0.051,0.079",0.111,2.04°,"-0.009,+0.005,-0.049",-0.81°
1->3,"0.067,0.054,0.037",0.093,1.58°,"-0.006,-0.009,-0.021",-0.15°
1->4,"0.024,0.023,0.031",0.045,1.10°,"-0.004,+0.007,-0.008",+0.09°
1->5,"0.049,0.052,0.028",0.077,1.09°,"0.007,+0.008,+0.006",+0.13°
Avg:,,0.082,1.45°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,114.8m,0.086,1.625,9.4e-05,8.8e-05,3.7e-05,nan,nan°
2by1,108.1m,0.082,1.103,8.7e-05,9.3e-05,1.9e-05,nan,nan°
3by1,112.2m,0.322,5.454,3.9e-04,1.9e-03,2.3e-05,nan,nan°
4by1,67.4m,0.154,3.023,6.3e-04,7.2e-04,1.0e-04,nan,nan°
5by1,105.4m,0.237,3.929,9.6e-04,1.1e-03,5.9e-04,nan,nan°
Avg.,101.6,0.176,3.03,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.063,0.066,0.040",0.099,1.74°,"-0.008,+0.023,-0.027",-0.89°
1->3,"0.210,0.214,0.036",0.302,4.74°,"0.046,-0.090,-0.032",+3.75°
1->4,"0.054,0.065,0.040",0.094,1.12°,"-0.003,-0.027,+0.037",-0.28°
1->5,"0.109,0.087,0.153",0.207,3.59°,"-0.014,-0.014,+0.076",-2.32°
Avg:,,0.176,2.80°,,
